In [1]:
import logging
import os
from datetime import datetime

import numpy as np
import torch

import ns3ai_gym_env
import gymnasium as gym

from sim_args import TcpRlSimArgs
from custom.model import *
from custom.wrapper import CustomMonitor
from custom.callback import TensorboardCallback
from custom.environment import Ns3EnvWrapped

In [ ]:
    parser.add_argument(
        "--seed", default=42, type=int, help="set seed for reproducibility"
    )
    parser.add_argument(
        "--sim_seed", default=0, type=int, help="set simulation run number"
    )
    parser.add_argument(
        "--duration", type=float, default=100, help="set simulation duration (seconds)"
    )
    parser.add_argument(
        "--save_dir",
        type=str,
        default="save",
        help="Location where to save data",
    )
    parser.add_argument(
        "--load_model", default=None, type=str, help="Load model from file, optional"
    )
    parser.add_argument(
        "--tensorboard_log",
        action="store_true",
        help="Log to Tensorboard, optional",
    )
    parser.add_argument(
        "--log_level",
        default="info",
        type=str.upper,
        choices=["CRITICAL", "FATAL", "ERROR", "WARN", "WARNING", "INFO", "DEBUG"],
        help="Log level (CRITICAL, FATAL, ERROR, WARN, WARNING, INFO, DEBUG)",
    )

In [2]:
# set seed for python libraries, optional
my_seed = 0

np.random.seed(my_seed)
torch.manual_seed(my_seed)

In [3]:
# create env with env_kwargs passed as ns3 arguments
env_kwargs = {"ns3Settings": TcpRlSimArgs(duration=10, simSeed=0).asdict()}

env = gym.make(
    "ns3ai_gym_env/Ns3-v0", targetName="rl_tcp_gym", ns3Path="../../", **env_kwargs  # type: ignore
)

# wrap env so seed will be changed after each reset
env = Ns3EnvWrapped(env=env)

ns3ai_utils: Experiment initialized
ns3ai_utils: Running ns-3 with:  ./ns3 run rl_tcp_gym -- --simSeed=0 --envTimeStep=0.1 --nLeaf=1 --error_p=0 --bottleneck_bandwidth=2Mbps --bottleneck_delay=0.01ms --access_bandwidth=10Mbps --access_delay=20ms --prefix_name=TcpVariantsComparison --data=1 --mtu=400 --duration=10.0 --flow_monitor=False --queue_disc_type=ns3::PfifoFastQueueDisc --sack=True --recovery=ns3::TcpClassicRecovery --transport_prot=TcpRlTimeBased
[0/2] Re-checking globbed directories...
ninja: no work to do.


C++ side random seed: 0
Tcp version: ns3::TcpRlTimeBased


Command 'build/scratch/reinforcement-learning/ns3.40-rl_tcp_gym-debug --simSeed=0 --envTimeStep=0.1 --nLeaf=1 --error_p=0 --bottleneck_bandwidth=2Mbps --bottleneck_delay=0.01ms --access_bandwidth=10Mbps --access_delay=20ms --prefix_name=TcpVariantsComparison --data=1 --mtu=400 --duration=10.0 --flow_monitor=False --queue_disc_type=ns3::PfifoFastQueueDisc --sack=True --recovery=ns3::TcpClassicRecovery --transport_prot=TcpRlTimeBased' died with <Signals.SIGTERM: 15>.


In [ ]:
ob_space = env.observation_space
ac_space = env.action_space
logging.info(f"Observation space: {ob_space}, {ob_space.dtype}")
logging.info(f"Action space: {ac_space}, {ac_space.dtype}")

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 13556), started 0:01:56 ago. (Use '!kill 13556' to kill it.)